In [2]:
!pip install scikeras

In [3]:
!pip install scikit-learn==1.5.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 49.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.5.0 which is incompatible.


In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.optimizers import RMSprop, Adam, SGD
from keras.metrics import binary_crossentropy
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier

In [7]:
df = pd.read_csv('train_minmax.csv')

In [6]:
df.head()

,age,demogweight,education-num,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,...,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Unknown,native-country_Vietnam,native-country_Yugoslavia,<=50K.,>50K.
0,0.360656,0.161894,0.739130,0.0,0.0,0.375,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
1,0.540984,0.176277,0.739130,0.0,0.0,0.000,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
2,0.344262,0.504714,0.391304,0.0,0.0,0.375,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0
3,0.590164,0.552056,0.217391,0.0,0.0,0.375,0,0,0,1,...,0,0,0,0,1,0,0,0,1,0
4,0.180328,0.809395,0.739130,0.0,0.0,0.375,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0


In [8]:
df.describe()

,age,demogweight,education-num,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,...,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Unknown,native-country_Vietnam,native-country_Yugoslavia,<=50K.,>50K.
count,25000.000000,25000.000000,25000.000000,25000.0,25000.0,25000.000000,25000.000000,25000.00000,25000.000000,25000.000000,...,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000,25000.000000
mean,0.353744,0.433172,0.488951,0.0,0.0,0.434176,0.030000,0.06496,0.000200,0.695400,...,0.002560,0.001680,0.000600,0.000480,0.896840,0.017800,0.002000,0.000440,0.760640,0.239360
std,0.222878,0.235655,0.212980,0.0,0.0,0.308885,0.170591,0.24646,0.014141,0.460247,...,0.050533,0.040954,0.024488,0.021904,0.304174,0.132227,0.044678,0.020972,0.426701,0.426701
min,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.180328,0.262278,0.391304,0.0,0.0,0.375000,0.000000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,0.327869,0.412158,0.478261,0.0,0.0,0.375000,0.000000,0.00000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,0.508197,0.557367,0.652174,0.0,0.0,0.625000,0.000000,0.00000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,0.0,0.0,1.000000,1.000000,1.00000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
# for c in df.columns:
#   print(c)

In [10]:
df.columns[101]

'<=50K.'

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    df.iloc[:,0:100],
    df.iloc[:,[-2,-1]],
    test_size=0.2,
    random_state=42,
    stratify=df.iloc[:,[-2,-1]]
)

In [11]:
# model = Sequential()
# model.add(Input(shape=(100,)))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(
#     optimizer=RMSprop(),
#     loss='binary_crossentropy',
#     metrics=['accuracy']
# )

In [12]:
y_train_single = np.argmax(y_train, axis=1)
# history = model.fit(X_train, y_train_single, batch_size=32, epochs=1, verbose=1, validation_split=0.2)

In [13]:
# val_acc = history.history['val_accuracy']
# val_loss = history.history['val_loss']

In [14]:
# plt.plot(val_acc, label='Validation Accuracy')
# plt.plot(val_loss, label='Validation Loss')
# plt.legend()
# plt.show()

In [39]:
y_test_single = np.argmax(y_test, axis=1)
# model.evaluate(X_test, y_test_single)

In [16]:
# preds = model.predict(X_test)

In [17]:
# preds

In [18]:
# decision_boundary = 0.5
# preds2 = [1 if x[0] > decision_boundary else 0 for x in preds]
# # preds = np.argmax(preds, axis=1)

# confusion_matrix(y_test_single, preds2)
# # confusion_matrix(np.argmax(y_test, axis=1), preds)

In [19]:
# fpr, tpr, thresholds = roc_curve(y_test_single, preds2)
# roc_auc = auc(fpr, tpr)

In [20]:
# plt.figure()
# plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {roc_auc:.4f})')
# plt.plot([0, 1], [0, 1], color='gray', linestyle='--')  # random guess line
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('ROC Curve')
# plt.legend()
# plt.show()

In [21]:
# boundaries = np.arange(0.05,0.95,0.05)
# for b in boundaries:
#     preds2 = [1 if x[0] > b else 0 for x in preds]
#     fpr, tpr, thresholds = roc_curve(y_test_single, preds2)
#     roc_auc = auc(fpr, tpr)
#     print(b, ":", roc_auc)


In [18]:
country_col = df.columns.str.contains('native')
df_no_country = df.loc[:,
       ~country_col]

In [24]:
df_no_country.columns[59]

'<=50K.'

In [33]:
X_train, X_test, y_train, y_test = train_test_split(
    df_no_country.iloc[:,0:59],
    df_no_country.iloc[:,[59,60]],
    test_size=0.2,
    random_state=42,
    stratify=df_no_country.iloc[:,[59,60]]
)

In [41]:
model = Sequential()
model.add(Input(shape=(59,)))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(64, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
model.compile(
    optimizer=RMSprop(learning_rate=0.01, momentum=0.01),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [42]:
y_train_single = np.argmax(y_train, axis=1)
history = model.fit(X_train, y_train_single, batch_size=32, epochs=15, verbose=1, validation_split=0.2)

Epoch 1/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7952 - loss: 0.4230 - val_accuracy: 0.8260 - val_loss: 0.3877
Epoch 2/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8305 - loss: 0.3626 - val_accuracy: 0.8320 - val_loss: 0.3554
Epoch 3/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8297 - loss: 0.3584 - val_accuracy: 0.8320 - val_loss: 0.3530
Epoch 4/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8354 - loss: 0.3512 - val_accuracy: 0.8322 - val_loss: 0.3568
Epoch 5/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8344 - loss: 0.3498 - val_accuracy: 0.8328 - val_loss: 0.3514
Epoch 6/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8327 - loss: 0.3533 - val_accuracy: 0.8267 - val_loss: 0.3595
Epoch 7/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8361 - loss: 0.3530 - val_accuracy: 0.8360 - val_loss: 0.3491
Epoch 8/15
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8360 - loss: 0.3520 - val_accuracy: 0.

In [44]:
model.evaluate(X_test, y_test_single)

157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8357 - loss: 0.3536


[0.35119563341140747, 0.8424000144004822]

In [43]:
preds = model.predict(X_test)
boundaries = np.arange(0.05,0.95,0.05)
for b in boundaries:
    preds2 = [1 if x[0] > b else 0 for x in preds]
    fpr, tpr, thresholds = roc_curve(y_test_single, preds2)
    roc_auc = auc(fpr, tpr)
    print(b, ":", roc_auc)

157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
0.05 : 0.7284941471041088
0.1 : 0.7633952529671975
0.15000000000000002 : 0.7798049334924655
0.2 : 0.7966473726607695
0.25 : 0.8057711110979306
0.3 : 0.8057585896549596
0.35000000000000003 : 0.8039287015856761
0.4 : 0.8000801152675712
0.45 : 0.7853244734238963
0.5 : 0.7675914740835788
0.55 : 0.745992314470109
0.6000000000000001 : 0.7215239870207555
0.6500000000000001 : 0.7014108151437406
0.7000000000000001 : 0.6811278349260828
0.7500000000000001 : 0.6550818056623723
0.8 : 0.6294151541532418
0.8500000000000001 : 0.6047153996833613
0.9000000000000001 : 0.5675925065534377


In [45]:
df_test = pd.read_csv('test_outliers_minmax.csv')

In [48]:
df_test.columns

Index(['age', 'demogweight', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week', 'workclass_Federal-gov', 'workclass_Local-gov',
       'workclass_Never-worked', 'workclass_Private', 'workclass_Self-emp-inc',
       'workclass_Self-emp-not-inc', 'workclass_State-gov',
       'workclass_Unknown', 'workclass_Without-pay', 'education_10th',
       'education_11th', 'education_12th', 'education_1st-4th',
       'education_5th-6th', 'education_7th-8th', 'education_9th',
       'education_Assoc-acdm', 'education_Assoc-voc', 'education_Bachelors',
       'education_Doctorate', 'education_HS-grad', 'education_Masters',
       'education_Preschool', 'education_Prof-school',
       'education_Some-college', 'marital-status_Divorced',
       'marital-status_Married-AF-spouse', 'marital-status_Married-civ-spouse',
       'marital-status_Married-spouse-absent', 'marital-status_Never-married',
       'marital-status_Separated', 'marital-status_Widowed',
       'occupation_Adm-c

In [51]:
df_test2 = df_test.loc[
    :,
    ~country_col[:-3]
]

In [53]:
preds_test = model.predict(df_test2)

237/237 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [54]:
preds_test_values = [1 if x[0] > 0.25 else 0 for x in preds_test]
preds_test_values

[0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,


In [56]:
with open('results.txt', 'w') as f:
    for item in preds_test_values:
        f.write("{}\n".format(item))

CV Fine-tuning

In [22]:
def build_model(optimizer = "RMSprop", lr = 1e-3, units=64, act="sigmoid", layers=2, momentum=1e-3):
    opt_name = optimizer.lower()
    if opt_name == "rmsprop":
        opt = tf.keras.optimizers.RMSprop(learning_rate=lr, momentum=momentum)
    elif opt_name == "sgd":
        opt = tf.keras.optimizers.SGD(learning_rate=lr, momentum=momentum)
    elif opt_name == "adam":
        opt = tf.keras.optimizers.Adam(learning_rate=lr, momentum=momentum)
    else:
        opt = tf.keras.optimizers.RMSprop(learning_rate=lr, momentum=momentum)

    model = Sequential()
    model.add(Input(shape=(100,)))
    for i in range(layers):
      model.add(Dense(units, activation=act))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=opt,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    return model

In [27]:
class CleanKerasClassifier(KerasClassifier):
    def fit(self, X, y, **kwargs):
        result = super().fit(X, y, **kwargs)
        tf.keras.backend.clear_session()
        gc.collect()
        return result

clf = CleanKerasClassifier(model=build_model, epochs=10, batch_size=32, verbose=0)

In [28]:
#clf = KerasClassifier(model=build_model, epochs=10, batch_size=32, verbose=1)
param_grid = {
    #"model__units": [32, 64],
    #"model__layers": [1, 2, 3],
    #"model__optimizer": ["RMSprop", "sgd", "adam"],
    "model__lr": [1e-2, 1e-3, 1e-4],
    #"model__act": ["sigmoid", "relu", "tanh"],
    "model__momentum": [1e-2, 1e-3, 1e-4]
}

In [29]:
grid = GridSearchCV(clf, param_grid=param_grid, cv=4, scoring="accuracy")

In [30]:
#grid.fit(X_train, y_train_single)

Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8000 - loss: 0.4286
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8310 - loss: 0.3583
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8357 - loss: 0.3512
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8357 - loss: 0.3579
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8416 - loss: 0.3495
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8415 - loss: 0.3462
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8359 - loss: 0.3525
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8441 - loss: 0.3379
Epoch 9/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8469 - loss: 0.3337
Epoch 10/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8441 - loss: 0.3404
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8025 - loss: 0.4182
Epoch 2/10
4

GridSearchCV(cv=4,
             estimator=CleanKerasClassifier(batch_size=32, epochs=10, model=<function build_model at 0x7ff594344720>),
             param_grid={'model__lr': [0.01, 0.001, 0.0001],
                         'model__momentum': [0.01, 0.001, 0.0001]},
             scoring='accuracy')

In [31]:
# print("Best params:", grid.best_params_)
# print("Best CV Acc:", grid.best_score_)

Best params: {'model__lr': 0.01, 'model__momentum': 0.01}
Best CV Acc: 0.83665


In [32]:
# model = Sequential()
# model.add(Input(shape=(100,)))
# for i in range(2):
#   model.add(Dense(64, activation="sigmoid"))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(
#     optimizer=RMSprop(learning_rate=0.01, momentum=0.01),
#     loss='binary_crossentropy',
#     metrics=['accuracy']
# )

In [33]:
# history = model.fit(X_train, y_train_single, batch_size=32, epochs=20, verbose=1, validation_split=0.2)

Epoch 1/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8026 - loss: 0.4186 - val_accuracy: 0.8232 - val_loss: 0.3722
Epoch 2/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8265 - loss: 0.3645 - val_accuracy: 0.8322 - val_loss: 0.3525
Epoch 3/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8296 - loss: 0.3604 - val_accuracy: 0.8332 - val_loss: 0.3531
Epoch 4/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8360 - loss: 0.3504 - val_accuracy: 0.8282 - val_loss: 0.3568
Epoch 5/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8340 - loss: 0.3563 - val_accuracy: 0.8317 - val_loss: 0.3574
Epoch 6/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8372 - loss: 0.3487 - val_accuracy: 0.8310 - val_loss: 0.3600
Epoch 7/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8357 - loss: 0.3516 - val_accuracy: 0.8325 - val_loss: 0.3500
Epoch 8/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8298 - loss: 0.3513 - val_accuracy: 0.

In [34]:
# model.evaluate(X_test, y_test_single)

157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8403 - loss: 0.3467


[0.34611913561820984, 0.8456000089645386]

In [35]:
# preds = model.predict(X_test)
# boundaries = np.arange(0.05,0.95,0.05)
# for b in boundaries:
#     preds2 = [1 if x[0] > b else 0 for x in preds]
#     fpr, tpr, thresholds = roc_curve(y_test_single, preds2)
#     roc_auc = auc(fpr, tpr)
#     print(b, ":", roc_auc)

157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
0.05 : 0.7625214978896975
0.1 : 0.7904228974575102
0.15000000000000002 : 0.8005518661233677
0.2 : 0.8116326841294664
0.25 : 0.8066473924314687
0.3 : 0.8078496706311311
0.35000000000000003 : 0.7938828796946349
0.4 : 0.7797778036993616
0.45 : 0.7649537332682218
0.5 : 0.7496585710045998
0.55 : 0.7328021825094774
0.6000000000000001 : 0.7023160935031065
0.6500000000000001 : 0.6826904890414307
0.7000000000000001 : 0.6695551658531024
0.7500000000000001 : 0.6483052885961947
0.8 : 0.6147021291505562
0.8500000000000001 : 0.584239325634623
0.9000000000000001 : 0.5402166780787537
